**Atividade Somativa 2** - Processamento de Linguagem

# Classificação de textos - Análise de Sentimentos
/content/drive/My Drive/PLN-Somativa2/analise-sentimentos-2000-noticias.txt


In [69]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
!ls "/content/drive/My Drive/PLN-Somativa2"


analise-sentimentos-2000-noticias.txt  kaggle.json


In [82]:
f = open('/content/drive/My Drive/PLN-Somativa2/analise-sentimentos-2000-noticias.txt', 'r', encoding='utf-8-sig')
linhas = f.readlines()


In [84]:
# Abrindo o arquivo salvo no Google Drive
arquivo_caminho = '/content/drive/My Drive/PLN-Somativa2/analise-sentimentos-2000-noticias.txt'
f = open(arquivo_caminho, 'r', encoding='utf-8-sig')
linhas = f.readlines()

# Exibindo as primeiras linhas para verificar se foi aberto corretamente
print(linhas[:5])  # Mostra as 5 primeiras linhas


['alegria;;irã liberta viúva condenada ao apedrejamento, diz comitê internacional. a viúva iraniana sakineh mohammadi-ashtiani, condenada à morte por apedrejamento, foi libertada juntamente com o seu filho e o seu advogado, anunciou nesta quinta-feira (9) o comitê internacional contra apedrejamento, sediado na alemanha.\n', 'tristeza;;haiti sofre com cólera depois da passagem do furacão sandy. após a passagem do furacão sandy pelo haiti, o país vive as dificuldades da reconstrução, além de problemas causados pela contaminação da bactéria que causa o cólera.\n', 'surpresa;;mulher acorda e encontra crocodilo de 3 m e 27 ovos no jardim de casa. especialistas capturaram e transferiram animal para local adequado.\n', 'tristeza;;enchentes causam a morte de 4 pessoas em al: vítimas são mãe, filho, idosa de 100 anos e adolescente. defesa civil da capital já registrou mais de 30 ocorrências.\n', 'neutro;;sem detalhes do que será discutido, revisão do plano diretor começa na segunda (17). dez of

In [85]:
# Criando listas para armazenar textos e suas respectivas classes
corpus_textos = []
classes = []

# Processando as linhas
for linha in linhas:
    # Dividindo a linha em texto e classe
    classe, texto = linha.strip().split(';;')
    corpus_textos.append(texto)
    classes.append(classe)

# Exibindo os primeiros textos e classes para verificar
print(corpus_textos[:5])
print(classes[:5])


['irã liberta viúva condenada ao apedrejamento, diz comitê internacional. a viúva iraniana sakineh mohammadi-ashtiani, condenada à morte por apedrejamento, foi libertada juntamente com o seu filho e o seu advogado, anunciou nesta quinta-feira (9) o comitê internacional contra apedrejamento, sediado na alemanha.', 'haiti sofre com cólera depois da passagem do furacão sandy. após a passagem do furacão sandy pelo haiti, o país vive as dificuldades da reconstrução, além de problemas causados pela contaminação da bactéria que causa o cólera.', 'mulher acorda e encontra crocodilo de 3 m e 27 ovos no jardim de casa. especialistas capturaram e transferiram animal para local adequado.', 'enchentes causam a morte de 4 pessoas em al: vítimas são mãe, filho, idosa de 100 anos e adolescente. defesa civil da capital já registrou mais de 30 ocorrências.', 'sem detalhes do que será discutido, revisão do plano diretor começa na segunda (17). dez oficinas, 19 audiências públicas e canal interativo serão

# 1) Redução da granularidade dos sentimentos.

Reduzimos as sete emoções do nosso corpus para três categorias: positivo, neutro e negativo. A classe "positivo" inclui "alegria", o "negativo" inclui "raiva", "medo", "desgosto" e "tristeza", e "neutro" permaneceu igual. Deletamos a emoção "surpresa". Notamos a quantidade de exemplos e encontramos 1022 negativos, 542 neutros e 184 positivos.

Com a nova base de dados, treinamos um modelo usando Naive Bayes. O relatório de classificação mostrou que o modelo foi eficaz em identificar sentimentos. Para a classe "positivo", teve uma precisão de 0.86, o que é bom. Para "neutro", a precisão foi de 0.66, o que significa que pode melhorar. Já para "negativo", a precisão foi de 0.70, mas a capacidade de identificar corretamente esses sentimentos foi baixa (0.21). No geral, a precisão total do modelo foi de 0.79, indicando que a redução das emoções ajudou, mas ainda precisamos melhorar a identificação dos sentimentos negativos.

In [94]:
# Criar um novo corpus com as classes reduzidas
corpus_reduzido = []

# Percorre cada linha do arquivo original
for linha in linhas:
    sentimento = linha.split(";;")[0]  # Pega o sentimento
    texto = linha.split(";;")[1]  # Pega o texto

    # Condições para criar as novas classes
    if sentimento == "alegria":
        corpus_reduzido.append("positivo;;" + texto)
    elif sentimento in ["raiva", "medo", "desgosto", "tristeza"]:
        corpus_reduzido.append("negativo;;" + texto)
    elif sentimento == "neutro":
        corpus_reduzido.append("neutro;;" + texto)

# Salvar a nova base de dados no arquivo corpus_reduzido.txt
with open("corpus_reduzido.txt", "w", encoding="utf-8-sig") as f:
    for item in corpus_reduzido:
        f.write(item)


In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Carregar o corpus reduzido
with open("corpus_reduzido.txt", "r", encoding="utf-8-sig") as f:
    linhas = f.readlines()

# Separar os textos e os rótulos
corpus_textos = [linha.split(";;")[1] for linha in linhas]
corpus_sentimentos = [linha.split(";;")[0] for linha in linhas]

# Alterando parâmetros do CountVectorizer: min_df = 2 para ignorar palavras muito raras
vectorizer = CountVectorizer(min_df=2, ngram_range=(1,2))  # Usando bigramas

# Vetorizar o corpus
X = vectorizer.fit_transform(corpus_textos)

# Separar dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, corpus_sentimentos, test_size=0.2, random_state=42)

# Utilizar o classificador MultinomialNB (Naive Bayes)
clf = MultinomialNB()

# Treinar o classificador
clf.fit(X_train, y_train)

# Fazer previsões
y_pred = clf.predict(X_test)

# Avaliar os resultados
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negativo       0.89      0.81      0.85       219
      neutro       0.60      0.84      0.70        97
    positivo       0.62      0.29      0.40        34

    accuracy                           0.77       350
   macro avg       0.71      0.65      0.65       350
weighted avg       0.79      0.77      0.77       350



In [96]:
# Carregar o novo corpus com a granularidade reduzida
with open("corpus_reduzido.txt", "r", encoding="utf-8-sig") as f:
    linhas = f.readlines()

# Separar os textos e os rótulos
corpus_textos = [linha.split(";;")[1] for linha in linhas]
corpus_sentimentos = [linha.split(";;")[0] for linha in linhas]

# Visualizar a quantidade de exemplos em cada classe (positivo, neutro, negativo)
from collections import Counter
print(Counter(corpus_sentimentos))

# Exemplo de como as primeiras linhas do corpus foram distribuídas
for i in range(5):
    print(f"Texto: {corpus_textos[i]} -> Sentimento: {corpus_sentimentos[i]}")


Counter({'negativo': 1022, 'neutro': 542, 'positivo': 184})
Texto: irã liberta viúva condenada ao apedrejamento, diz comitê internacional. a viúva iraniana sakineh mohammadi-ashtiani, condenada à morte por apedrejamento, foi libertada juntamente com o seu filho e o seu advogado, anunciou nesta quinta-feira (9) o comitê internacional contra apedrejamento, sediado na alemanha.
 -> Sentimento: positivo
Texto: haiti sofre com cólera depois da passagem do furacão sandy. após a passagem do furacão sandy pelo haiti, o país vive as dificuldades da reconstrução, além de problemas causados pela contaminação da bactéria que causa o cólera.
 -> Sentimento: negativo
Texto: enchentes causam a morte de 4 pessoas em al: vítimas são mãe, filho, idosa de 100 anos e adolescente. defesa civil da capital já registrou mais de 30 ocorrências.
 -> Sentimento: negativo
Texto: sem detalhes do que será discutido, revisão do plano diretor começa na segunda (17). dez oficinas, 19 audiências públicas e canal intera

In [97]:
# Importar bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Vetorização do texto
vectorizer = CountVectorizer()

# Transformar o corpus em uma matriz de contagem de palavras
X = vectorizer.fit_transform(corpus_textos)

# Separar os dados em treinamento e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, corpus_sentimentos, test_size=0.2, random_state=42)

# Treinamento do classificador Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Previsão no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["positivo", "neutro", "negativo"]))

# Comparação dos resultados com a versão original (se você tiver o código ou resultados do modelo anterior)


Relatório de Classificação:
              precision    recall  f1-score   support

    positivo       0.86      0.87      0.87       219
      neutro       0.66      0.80      0.72        97
    negativo       0.70      0.21      0.32        34

    accuracy                           0.79       350
   macro avg       0.74      0.63      0.64       350
weighted avg       0.79      0.79      0.77       350



# 2) Configuração dos parâmetros de extração de atributos e do classificador

Nesta etapa, queríamos melhorar a classificação dos sentimentos usando nosso novo conjunto de dados com três emoções: positivo, neutro e negativo.

Primeiro, ajustamos como extraímos as palavras, usando o CountVectorizer, que limita as 1.000 palavras mais comuns. Assim treinamos um modelo de classificação utilizando o Naive Bayes e avaliamos seu desempenho, com os seguintes resultados: para a classe "negativo", a precisão foi de 0.86, para "neutro" foi de 0.62, e para "positivo" foi de 0.44. A precisão total do modelo foi de 0.75.

Depois, trocamos o CountVectorizer pelo TfidfVectorizer, que leva em conta a importância das palavras em cada texto. Usamos o Naive Bayes novamente para avaliar o modelo.Por fim, testamos um novo classificador chamado LogisticRegression, treinando e avaliando da mesma forma.

Essas etapas nos ajudaram a ver como diferentes métodos e configurações afetam a classificação dos sentimentos, permitindo melhorias no nosso modelo, já que os resultados mostram que, apesar de um desempenho razoável, ainda podemos aprimorar a identificação da classe "positivo".

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Configuração do CountVectorizer
vectorizer = CountVectorizer(max_features=1000)  # Limitando a 1000 palavras

# Criar o pipeline com o classificador Naive Bayes
pipeline = make_pipeline(vectorizer, MultinomialNB())

# Separar os dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(corpus_textos, corpus_sentimentos, test_size=0.2, random_state=42)

# Treinamento do modelo
pipeline.fit(X_train, y_train)

# Avaliação do modelo
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negativo       0.86      0.81      0.83       219
      neutro       0.62      0.80      0.70        97
    positivo       0.44      0.21      0.28        34

    accuracy                           0.75       350
   macro avg       0.64      0.61      0.60       350
weighted avg       0.75      0.75      0.74       350



# 3. Adicione novas etapas de extração de atributos ou de pré-processamento

Nesta terceira etapa, usamos uma nova técnica chamada TF-IDF para ajudar o modelo a entender melhor quais palavras são importantes para analisar os sentimentos. A ideia era melhorar a precisão das previsões.

Depois de treinar o modelo, os resultados mostraram que ele acertou bastante os sentimentos negativos, com uma precisão de 0.69 e um recall de 0.97. Para os sentimentos neutros, a precisão foi de 0.75, mas o recall ficou baixo em 0.34, indicando que muitos sentimentos neutros não foram identificados. Já para os sentimentos positivos, o modelo não conseguiu prever nada, com precisão e recall de 0.00.

No total, a precisão do modelo foi de 0.70. Isso mostra que, embora conseguimos melhorar na identificação de sentimentos negativos e neutros, ainda temos dificuldades em classificar os sentimentos positivos.

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Separar os dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(corpus_textos, corpus_sentimentos, test_size=0.2, random_state=42)

# Criar o pipeline com TF-IDF
pipeline_tfidf = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Treinar o modelo
pipeline_tfidf.fit(X_train, y_train)

# Fazer previsões
y_pred = pipeline_tfidf.predict(X_test)

# Avaliar o modelo
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


Matriz de Confusão:
[[212   7   0]
 [ 64  33   0]
 [ 30   4   0]]

Relatório de Classificação:
              precision    recall  f1-score   support

    negativo       0.69      0.97      0.81       219
      neutro       0.75      0.34      0.47        97
    positivo       0.00      0.00      0.00        34

    accuracy                           0.70       350
   macro avg       0.48      0.44      0.43       350
weighted avg       0.64      0.70      0.64       350



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 4) Utilize outro classificador de texto

Na quarta etapa usamos um novo classificador, o Random Forest. Primeiro, dividimos os dados: uma parte para treinar o modelo e outra para testar. Transformamos os textos em um formato que o computador consegue entender e treinamos o modelo. Depois, o modelo fez previsões com os dados de teste. Ele acertou bem os sentimentos negativos, com uma precisão de 0.71 e recall de 0.95. Para os sentimentos neutros, a precisão foi de 0.72, eos positivos, não conseguiu prever nada, com precisão e recall de 0.00.
No total, a precisão do modelo foi de 0.71. Isso mostra que melhoramos na identificação de sentimentos negativos e neutros, mas ainda precisamos trabalhar mais para reconhecer os sentimentos positivos.

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Separar os dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(corpus_textos, corpus_sentimentos, test_size=0.2, random_state=42)

# Criar um pipeline com o classificador Random Forest
pipeline_rf = Pipeline([
    ('vectorizer', CountVectorizer()),  # Vetorização de palavras
    ('classifier', RandomForestClassifier(n_estimators=100))  # Classificador Random Forest
])

# Treinamento do modelo
pipeline_rf.fit(X_train, y_train)

# Previsão nos dados de teste
y_pred = pipeline_rf.predict(X_test)

# Relatório de classificação para visualizar resultados
print(classification_report(y_test, y_pred))

# Exemplos de previsões
for i in range(5):
    print(f"Texto: {X_test[i]} -> Sentimento Previsto: {y_pred[i]}")


              precision    recall  f1-score   support

    negativo       0.71      0.95      0.81       219
      neutro       0.72      0.44      0.55        97
    positivo       0.00      0.00      0.00        34

    accuracy                           0.71       350
   macro avg       0.48      0.46      0.45       350
weighted avg       0.65      0.71      0.66       350

Texto: vândalos entram em escola e queimam livros e documentos, no pr. caso aconteceu na madrugada desta segunda-feira (7), em cascavel. câmeras de segurança flagraram quatro pessoas entrando no colégio.
 -> Sentimento Previsto: negativo
Texto: merkel: violação do direito internacional causará reação. a chanceler da alemanha, angela merkel, disse nesta sexta-feira que é importante manter a porta aberta para discussões com a rússia sobre a ucrânia, mas que o ocidente precisa ser consistente na resposta a violações do direito internacional.
 -> Sentimento Previsto: neutro
Texto: verissimo continua em estado grave,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Nesta atividade, trabalhamos na classificação de sentimentos em quatro etapas. Primeiro, reduzimos as emoções para três categorias: positivo, neutro e negativo. Em seguida, melhoramos a forma como extraímos as palavras e usamos a técnica TF-IDF, que ajuda a encontrar as palavras mais importantes nos textos. Depois, testamos diferentes métodos para tentar melhorar a precisão das análises. Usamos um conjunto de dados do Kaggle, que nos deu informações reais para trabalhar. Embora tenhamos conseguido identificar melhor os sentimentos negativos e neutros, ainda temos dificuldades com os sentimentos positivos, o que mostra que precisamos continuar aprimorando nosso modelo.